In [1]:
import sys

sys.path.append('/home/rustambaku13/Documents/Warwick/flink-streaming-gnn/python')

In [66]:
import jax
from flax.linen import relu, softmax, sigmoid, Module
from nn.jax.multi_layer_dense import MultiLayerDense
import jax.numpy as jnp
import pandas as pd
import numpy as np

## Data Preprocessing


In [86]:
edges = pd.read_csv("../dataset/cora/edges.csv", header=None, names=["src", "dest"] )
vertices = pd.read_csv("../dataset/cora/group-edges.csv", header=None, names=['vertex', "label"])

unique_features = vertices.label.unique()
one_hot_features = jnp.eye(unique_features.shape[0])
label_feature_map = {k:one_hot_features[i] for i,k in enumerate(unique_features)}
vertices['feature'] = vertices.label.map(lambda x: label_feature_map[x])
vertices['train'] = np.random.rand(vertices.shape[0]) < 0.2
vertices['feature_train'] = vertices.label.map(lambda x: label_feature_map[x])
vertex_index_map = {vertex: index for (index,vertex) in enumerate(vertices.vertex.values)}

edge_list = jnp.vstack(edges.apply(lambda x: [vertex_index_map[x.src], vertex_index_map[x.dest]], axis=1))
fea

In [105]:
vertices['feature'] && vertices['train']

SyntaxError: invalid syntax (<ipython-input-105-cb73e4df1045>, line 1)

In [103]:
jnp.vstack(list(vertices.feature.values)) & vertices.train.values

ValueError: Incompatible shapes for broadcasting: ((2708, 7), (1, 2708))

In [76]:
vertex_index_map

{1000012: 0,
 100197: 1,
 100701: 2,
 100935: 3,
 100961: 4,
 101143: 5,
 101145: 6,
 101261: 7,
 101263: 8,
 101660: 9,
 101662: 10,
 10169: 11,
 10174: 12,
 10177: 13,
 101811: 14,
 10183: 15,
 10186: 16,
 102061: 17,
 1022969: 18,
 102406: 19,
 1026: 20,
 102879: 21,
 102884: 22,
 102938: 23,
 102939: 24,
 1031453: 25,
 1033: 26,
 1034: 27,
 103430: 28,
 103482: 29,
 1035: 30,
 103515: 31,
 103528: 32,
 103529: 33,
 103531: 34,
 103537: 35,
 103543: 36,
 10430: 37,
 10435: 38,
 104840: 39,
 105057: 40,
 1050679: 41,
 10531: 42,
 105856: 43,
 105865: 44,
 105899: 45,
 1059953: 46,
 1061127: 47,
 1063773: 48,
 106590: 49,
 107177: 50,
 1071981: 51,
 107251: 52,
 107252: 53,
 107569: 54,
 10793: 55,
 10796: 56,
 10798: 57,
 108047: 58,
 108962: 59,
 108963: 60,
 108974: 61,
 108983: 62,
 109323: 63,
 1095507: 64,
 10981: 65,
 110041: 66,
 110162: 67,
 110163: 68,
 110164: 69,
 1102364: 70,
 1102400: 71,
 1102407: 72,
 1102442: 73,
 1102548: 74,
 1102550: 75,
 1102567: 76,
 1102625: 77,

## Model Creation

In [ ]:
class GNNLayer(Module):
    message: Module
    update: Module
    agg: str
    
    def __call__(self, vertex_features, edges):
        
        